# Create predcitors with Prophet and DeepAR+

* **About 40 mins may be elapsed**

In [22]:
import boto3
from time import sleep
import pandas as pd
import json
import time
import pprint
import numpy as np

In [23]:
# 다른 노트북에서 저장된 변수들을 불러옴
%store -r

In [24]:
session = boto3.Session(region_name=region)
forecast = session.client(service_name='forecast')
forecast_query = session.client(service_name='forecastquery')

## Hyperparameters

In [25]:
forecastHorizon = 31 # the number of forecasting
NumberOfBacktestWindows = 2 # the number of Backtests
BackTestWindowOffset = 31 # the point from the end of the dataset where the data is split for model training and testing 
ForecastFrequency = "D"

아래 파라미터의 상세 정보는 [여기](https://docs.aws.amazon.com/forecast/latest/dg/metrics.html) 를 보세요.

#### Predictor algorithms

In [26]:
prophet_algorithmArn = 'arn:aws:forecast:::algorithm/Prophet'
deepAR_Plus_algorithmArn = 'arn:aws:forecast:::algorithm/Deep_AR_Plus'

### Prophet

In [27]:
# Prophet Specifics
prophet_predictorName= project+'_prophet_algo_1' + target_suffix + suffix

In [29]:
# Build Prophet:
prophet_create_predictor_response=forecast.create_predictor(
      PredictorName=prophet_predictorName, 
      AlgorithmArn=prophet_algorithmArn,
      ForecastHorizon=forecastHorizon,
      PerformAutoML= False,
      PerformHPO=False,
      EvaluationParameters= {"NumberOfBacktestWindows": NumberOfBacktestWindows, 
                             "BackTestWindowOffset": BackTestWindowOffset}, 
      InputDataConfig= {"DatasetGroupArn": target_datasetGroupArn, 
                        "SupplementaryFeatures": [ 
                         { 
                            "Name": "holiday",
                            "Value": "US"
                         }
                      ]},
      FeaturizationConfig= {"ForecastFrequency": ForecastFrequency, 
                            "ForecastDimensions":
                            ["store"],
                            "Featurizations": 
                            [
                              {"AttributeName": "target_value", 
                               "FeaturizationPipeline": 
                                [
                                  {"FeaturizationMethodName": "filling", 
                                   "FeaturizationMethodParameters": 
                                    {"frontfill": "none", 
                                     "middlefill": "zero", 
                                     "backfill": "zero"}
                                  }
                                ]
                              }
                            ]
                           }
                                                 )




### DeepAR+

In [30]:
# DeepAR+ Specifics
deeparp_predictorName= project+'_deeparp_algo_1' + suffix

In [31]:
# Build DeepAR+:
deeparp_create_predictor_response=forecast.create_predictor(
      PredictorName=deeparp_predictorName, 
      AlgorithmArn=deepAR_Plus_algorithmArn,
      ForecastHorizon=forecastHorizon,
      PerformAutoML= False,
      PerformHPO=False,
      EvaluationParameters= {"NumberOfBacktestWindows": NumberOfBacktestWindows, 
                             "BackTestWindowOffset": BackTestWindowOffset}, 
      InputDataConfig= {"DatasetGroupArn": target_datasetGroupArn, "SupplementaryFeatures": [ 
                         { 
                            "Name": "holiday",
                            "Value": "US"
                         }
                      ]},
      FeaturizationConfig= {"ForecastFrequency": ForecastFrequency, 
                            "ForecastDimensions":
                            ["store"],
                            
                            "Featurizations": 
                            [
                              {"AttributeName": "target_value", 
                               "FeaturizationPipeline": 
                                [
                                  {"FeaturizationMethodName": "filling", 
                                   "FeaturizationMethodParameters": 
                                    {"frontfill": "none", 
                                     "middlefill": "zero", 
                                     "backfill": "zero"}
                                  }
                                ]
                              }
                            ]
                           }
                                                 )





## Check if the predictors are active


In [32]:
# forecast.list_predictors(Filters=[
#     {"Condition": "IS",
#      "Key": "DatasetGroupArn", 
#      "Value": datasetGroupArn}
# ])

In [33]:
#prophet_predictor_arn = 'arn:aws:forecast:us-east-2:057716757052:predictor/WalmartKaggleGonsoo_prophet_algo_174655'
target_prophet_predictorArn = prophet_create_predictor_response["PredictorArn"]

In [34]:
target_deepar_predictorArn = deeparp_create_predictor_response["PredictorArn"]

In [35]:
%%time
# Check the Prophet status

while True:
    createProphetPredictorStatus = forecast.describe_predictor(PredictorArn= target_prophet_predictorArn)['Status']
    createDeeparpPredictorStatus = forecast.describe_predictor(PredictorArn= target_deepar_predictorArn)['Status']    
    print("Prophet: ", createProphetPredictorStatus)
    print("DeepARP: ", createDeeparpPredictorStatus)          
    if createProphetPredictorStatus != 'ACTIVE' and createProphetPredictorStatus != 'CREATE_FAILED':
        sleep(60)
    elif createDeeparpPredictorStatus != 'ACTIVE' and createDeeparpPredictorStatus != 'CREATE_FAILED':
        sleep(60)
    else:
        break

Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREA

## Predictor evaluation

#### Prophet

In [36]:
# Prophet Metrics
# prophet_arn = prophet_create_predictor_response['PredictorArn']
prophet_metrics = forecast.get_accuracy_metrics(PredictorArn=target_prophet_predictorArn)
pp = pprint.PrettyPrinter()
pp.pprint(prophet_metrics)

{'PredictorEvaluationResults': [{'AlgorithmArn': 'arn:aws:forecast:::algorithm/Prophet',
                                 'TestWindows': [{'EvaluationType': 'SUMMARY',
                                                  'Metrics': {'RMSE': 8.443304726264216,
                                                              'WeightedQuantileLosses': [{'LossValue': 0.05079870636900548,
                                                                                          'Quantile': 0.9},
                                                                                         {'LossValue': 0.10882895406442908,
                                                                                          'Quantile': 0.5},
                                                                                         {'LossValue': 0.04667087421210328,
                                                                                          'Quantile': 0.1}]}},
                                            

In [37]:
# deepar_predictorArn = deeparp_create_predictor_response["PredictorArn"]
# forecast.describe_predictor(PredictorArn= deepar_predictorArn)

#### DeepAR+

Same as Prophet and ARIMA, now you should look at the metrics from it.

In [38]:
# DeepAR+ Metrics
# deepar_predictor_arn = deeparp_create_predictor_response['PredictorArn']
deeparp_metrics = forecast.get_accuracy_metrics(PredictorArn=target_deepar_predictorArn)
pp = pprint.PrettyPrinter()
pp.pprint(deeparp_metrics)

{'PredictorEvaluationResults': [{'AlgorithmArn': 'arn:aws:forecast:::algorithm/Deep_AR_Plus',
                                 'TestWindows': [{'EvaluationType': 'SUMMARY',
                                                  'Metrics': {'RMSE': 10.002934740003255,
                                                              'WeightedQuantileLosses': [{'LossValue': 0.06869896346027182,
                                                                                          'Quantile': 0.9},
                                                                                         {'LossValue': 0.12819415419100966,
                                                                                          'Quantile': 0.5},
                                                                                         {'LossValue': 0.05075241882577648,
                                                                                          'Quantile': 0.1}]}},
                                      

In [40]:
%store target_prophet_predictorArn
%store target_deepar_predictorArn

Stored 'target_prophet_predictorArn' (str)
Stored 'target_deepar_predictorArn' (str)
